In [1]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from scipy import stats
import os

In [2]:
def load_data(file_name, entrystop=None, isEE=False):

    root_file = uproot.open(file_name)
#    root_file = uproot.xrootd(file_name)

    # The branches we need for the regression                                                                                                              
    branches_EB = [ 'clusterRawEnergy', 'full5x5_e3x3', 'full5x5_eMax',
            'full5x5_e2nd', 'full5x5_eTop', 'full5x5_eBottom', 'full5x5_eLeft',
            'full5x5_eRight', 'full5x5_e2x5Max', 'full5x5_e2x5Top',
            'full5x5_e2x5Bottom', 'full5x5_e2x5Left', 'full5x5_e2x5Right',
            'full5x5_e5x5', 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
            'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
            'full5x5_sigmaIphiIphi', 'iEtaSeed', 'iPhiSeed', 'iEtaMod5',
            'iPhiMod2', 'iEtaMod20', 'iPhiMod20', 'genEnergy']
    
    branches_EE = [ 'clusterRawEnergy', 'full5x5_e3x3', 'full5x5_eMax',
            'full5x5_e2nd', 'full5x5_eTop', 'full5x5_eBottom', 'full5x5_eLeft',
            'full5x5_eRight', 'full5x5_e2x5Max', 'full5x5_e2x5Top',
            'full5x5_e2x5Bottom', 'full5x5_e2x5Left', 'full5x5_e2x5Right',
            'full5x5_e5x5', 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
            'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
            'full5x5_sigmaIphiIphi',
            'genEnergy', 'iXSeed', 'iYSeed', 'preshowerEnergy']

    if isEE:
        branches = branches_EE + ["pt", "eta"]
    else:
        branches = branches_EB + ["pt", "eta"]
        
    df = root_file['een_analyzer/PhotonTree'].pandas.df(branches, entrystop=entrystop).dropna()
    print("Entries in ntuple:")
    print(len(df))

    # Define some ratio variables                                                                                                                          
    df.eval("clusertRawEnergyOverE5x5 = clusterRawEnergy/full5x5_e5x5", inplace=True)
    df.eval("w3x3OverE5x5             = full5x5_e3x3/full5x5_e5x5", inplace=True)
    df.eval("eMaxOverE5x5             = full5x5_eMax/full5x5_e5x5", inplace=True)
    df.eval("e2ndOverE5x5             = full5x5_e2nd/full5x5_e5x5", inplace=True)
    df.eval("eTopOverE5x5             = full5x5_eTop/full5x5_e5x5", inplace=True)
    df.eval("eBottomOverE5x5          = full5x5_eBottom/full5x5_e5x5", inplace=True)
    df.eval("eLeftOverE5x5            = full5x5_eLeft/full5x5_e5x5", inplace=True)
    df.eval("eRightOverE5x5           = full5x5_eRight/full5x5_e5x5", inplace=True)
    df.eval("e2x5MaxOverE5x5          = full5x5_e2x5Max/full5x5_e5x5", inplace=True)
    df.eval("e2x5TopOverE5x5          = full5x5_e2x5Top/full5x5_e5x5", inplace=True)
    df.eval("e2x5BottomOverE5x5       = full5x5_e2x5Bottom/full5x5_e5x5", inplace=True)
    df.eval("e2x5LeftOverE5x5         = full5x5_e2x5Left/full5x5_e5x5", inplace=True)
    df.eval("e2x5RightOverE5x5        = full5x5_e2x5Right/full5x5_e5x5", inplace=True)

    if isEE:
        df.eval("preshowerEnergyOverrawEnergy = preshowerEnergy/rawEnergy", inplace=True)

    # The target                                                                                                                                           
    if isEE:
        df.eval("target = genEnergy / ( rawEnergy + preshowerEnergy )", inplace=True)
    else:
        df.eval("target = genEnergy / rawEnergy", inplace=True)

    return df

        

In [3]:
# The features                                                                                                                                             
features_EB = [ 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
        'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
        'full5x5_sigmaIphiIphi', 'clusertRawEnergyOverE5x5', 'w3x3OverE5x5',
        'eMaxOverE5x5', 'e2ndOverE5x5', 'eTopOverE5x5', 'eBottomOverE5x5',
        'eLeftOverE5x5', 'eRightOverE5x5', 'e2x5MaxOverE5x5',
        'e2x5TopOverE5x5', 'e2x5BottomOverE5x5', 'e2x5LeftOverE5x5',
        'e2x5RightOverE5x5', 'iEtaSeed', 'iPhiSeed', 'iEtaMod5', 'iPhiMod2',
        'iEtaMod20', 'iPhiMod20']


# EE                                                                                                                                                       
features_EE = [ 'rawEnergy', 'etaWidth', 'phiWidth', 'rhoValue',
        'full5x5_sigmaIetaIeta', 'full5x5_sigmaIetaIphi',
        'full5x5_sigmaIphiIphi', 'clusertRawEnergyOverE5x5', 'w3x3OverE5x5',
        'eMaxOverE5x5', 'e2ndOverE5x5', 'eTopOverE5x5', 'eBottomOverE5x5',
        'eLeftOverE5x5', 'eRightOverE5x5', 'e2x5MaxOverE5x5',
        'e2x5TopOverE5x5', 'e2x5BottomOverE5x5', 'e2x5LeftOverE5x5',
        'e2x5RightOverE5x5', 'iXSeed', 'iYSeed', 'preshowerEnergyOverrawEnergy']

file_name = "/scratch/micheli/perfectIC-highpt-EB-training.root"

isEE = '-EE-' in file_name

if isEE:
    features = features_EE
else:
    features = features_EB

tmp = file_name.split("/")
out_dir = tmp[-2] + "_" + tmp[-1].replace("-training.root", "")

df_train = load_data(file_name, 400000,isEE=isEE)

Entries in ntuple:
648961


In [4]:
###train with default parameters
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
X = df_train[features]
y = df_train["target"]
xgtrain = GradientBoostingRegressor()

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=0)

#xgtrain.fit(X_train, y_train)

/t3home/micheli/.local/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/t3home/micheli/.local/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [7]:
### optimize parameters
from skopt import BayesSearchCV
# log-uniform: understand as search over p = exp(x) by varying x

#this is needed because of this https://www.kaggle.com/c/home-credit-default-risk/discussion/64004
#class BayesSearchCV(BayesSearchCV):
#    def _run_search(self, x): raise BaseException('Use newer skopt')

opt = BayesSearchCV(
    xgtrain,
    {
        'n_estimators':(50, 1000),
        'max_depth' :(1,10),
#        'learning_rate':(0.01,2.0,'log-uniform'),
#        'min_samples_leaf':(0.01,1,'log-uniform')
    },
#    cv=3,
#    pre_dispatch='2*n_jobs',
#    n_points=1, 
    n_jobs=10,
    n_iter=3,
    verbose=True
)

In [8]:
opt.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


Process PoolWorker-36:
Process PoolWorker-37:
Process PoolWorker-31:
Process PoolWorker-32:
Process PoolWorker-35:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-34:
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-33:
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
    self.run()
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    se

KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap


In [30]:
print ("train. score: %s" % xgtrain.score(X_train,y_train) )
print ("test. score: %s" % xgtrain.score(X_test,y_test) )
print "with optimization:"
print("val. score: %s" % opt.best_score_)
print("train score: %s" % opt.score(X_train, y_train))
print("test score: %s" % opt.score(X_test, y_test))

train. score: 0.9815817039164595
test. score: 0.8901812031904094
with optimization:
val. score: 0.7778014623071804
train score: 0.9469870777009766
test score: 0.9041111023646276


In [31]:
print opt.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=62, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


In [6]:
###tuning xgboost
xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [7]:
from skopt import BayesSearchCV

opt_xgb = BayesSearchCV(
    xgb_reg,
    {
        'n_estimators':(50, 1000),
        'max_depth' :(1,10),
#        'learning_rate':(0.01,2.0,'log-uniform'),
#        'min_samples_leaf':(0.01,1,'log-uniform')
    },
#    cv=3,
#    pre_dispatch='2*n_jobs',
    n_points=1, 
    n_jobs=10,
    n_iter=3,
    verbose=True
)

In [ ]:
opt_xgb.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


Process PoolWorker-6:
Process PoolWorker-8:
Process PoolWorker-4:
Process PoolWorker-9:
Process PoolWorker-7:
Process PoolWorker-10:
Process PoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/swshare/anaconda/lib/python2.7/mu